In [3]:
node_details = {
    'cores' :48,
    'ram' : 128
}

job_details = {
    'partition': 'ceewater_cjgleason-cpu',
    'number_things_to_process':50,
    'nodes':1,
    'cores':5,
    'ram':5,
    'time': 10000,
    'name': 'confluence-metroman',
    'run_command': 'python3 slurm_run_module.py -n metroman -e ${SLURM_ARRAY_TASK_ID}'
}



In [4]:


def construct_sh_dict(node_details, job_details):

    ram_limited = False
    cpu_limited = False

    sh_dict = {
        'partition': job_details['partition'],
        'nodes': job_details['nodes'],
        'ntasks-per-node':1,
        'cpus-per-task':job_details['cores'],
        'mem':str(job_details['ram'])+'G',
        'time': job_details['time'],
        'job-name': job_details['name'],
        'array': '0-' + str(job_details['number_things_to_process']),
        'run_command': job_details['run_command']

    }

    resources_available = {
        'cores': node_details['cores'] * job_details['nodes'],
        'ram': node_details['ram'] * job_details['nodes']
    }

    ram_constraint = resources_available['ram']/job_details['ram']
    cpu_constraint = resources_available['cores']/job_details['cores']
    print(ram_constraint, cpu_constraint)

    if ram_constraint > cpu_constraint:
        cpu_limited = True

    else:
        ram_limited = True

    if cpu_limited:
        array_mod = cpu_constraint
    
    if ram_limited:
        array_mod = ram_constraint

    if array_mod < job_details['number_things_to_process']:
        sh_dict['array'] += f'%{str(int(array_mod))}'

    return sh_dict

def create_slurm_script(node_details=node_details, job_details=job_details):
    sh_dict = construct_sh_dict(node_details=node_details, job_details=job_details)
    file = open('items.sh','w')
    file.write('#!/bin/bash \n')
    file.write('#SBATCH -o ./Report/output.%a.out \n')

    for item in sh_dict:
        if item != 'run_command':
            file.write(f'#SBATCH --{item}={sh_dict[item]} \n')
        else:
            file.write('\nmodule load singularity \n')
            file.write(f'{sh_dict[item]}')
    file.close()


create_slurm_script(node_details=node_details, job_details=job_details)


25.6 9.6
